### This challenge presents a problem in classification because the goal of the problem is to find which features predict the adopted users which is a nominal variable. 

### We will build a random forest classifer to predict the features but firstly we have to examine the data set and clean the data to a standard format for the model. 

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
user_df = pd.read_csv("takehome_users.csv")
user_df.head(3)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0


In [34]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


Eventhough there are two features that haves missing values: last_session_creation_time and invited_by_user_id, we won't try to solve this issue since they both are not so relevant for the sake of this study. Yet the last session creation time of 8823 is the users who actually used the application where 12000 signed up for the program. 

In [35]:
user_df['creation_time']= pd.to_datetime(user_df['creation_time'])

In [36]:
user_df.head(3)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0


In [37]:
user_engage_df = pd.read_csv("takehome_user_engagement.csv")
user_engage_df.head(3)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1


In [38]:
user_engage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [39]:
user_engage_df['time_stamp'] = pd.to_datetime(user_engage_df['time_stamp'])

In [40]:
import datetime
from collections import Counter
#creating a column for the adapted users who login 3 separate days within 1 week. 
#create a time period of 7 days and find true or false by determine the 3-day signin is within the one week
days7 = datetime.timedelta(7)
adopt_dict = {}

for user_id in sorted(list(user_engage_df['user_id'].unique())):
    adopt_user = False
    temp = user_engage_df[user_engage_df['user_id'] == user_id]
    temp = temp.sort_values('time_stamp')
    for row in temp.itertuples():
        if adopt_user == True:
            continue
        this_time = temp.get_value(row[0], 'time_stamp')
        if len(temp[(temp['time_stamp'] >= this_time) & (temp['time_stamp'] <= (this_time + days7))]) >= 3:
            adopt_user = True
    adopt_dict[user_id] = adopt_user

In [41]:
# replace False with the correct flag of adapted users  
user_df['adopted'] = False

for row in (user_df[np.logical_not(user_df['last_session_creation_time'].isnull())]).itertuples(): 
    adopt_value = adopt_dict[user_df.get_value(row[0], 'object_id')]
    user_df.set_value(row[0], 'adopted', adopt_value)

In [42]:
user_df.head(3)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,True
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,False


In [43]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null datetime64[ns]
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
adopted                       12000 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 949.3+ KB


In [46]:
user_df.groupby('adopted')['name'].count()

adopted
False    10344
True      1656
Name: name, dtype: int64

In [72]:
#select only relevant features
select_user_df = user_df[['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id','adopted']]

In [73]:
#develop a model for a classifer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [74]:
decode_df = pd.get_dummies(select_user_df)
decode_df.head(3)

,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1,0,11,False,1,0,0,0,0
1,0,0,1,True,0,1,0,0,0
2,0,0,94,False,0,1,0,0,0


In [75]:
#split the data
#split the dataset into train and test
train, test = train_test_split(decode_df, test_size = .30, stratify=decode_df['adopted'])
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
print train.shape
print test.shape

(8400, 9)
(3600, 9)


In [76]:
target = train['adopted']
train = train.drop('adopted', axis=1)
target_test = test['adopted']
test = test.drop('adopted', axis=1)

In [79]:
clf = RandomForestClassifier()
fit_tree= clf.fit(train, target)
fit_tree.predict(test)
print ("Accurary Score: %.4f")%clf.score(test, target_test)

Accurary Score: 0.8144


In [83]:
feature_importance = pd.DataFrame(fit_tree.feature_importances_).T
feature_importance.columns = list(train.columns)
feature_importance = feature_importance.T
feature_importance.columns = ['feature_importance']
feature_importance.sort_values('feature_importance', ascending=False)

,feature_importance
org_id,0.965929
enabled_for_marketing_drip,0.008905
creation_source_PERSONAL_PROJECTS,0.008423
opted_in_to_mailing_list,0.006964
creation_source_GUEST_INVITE,0.004110
creation_source_SIGNUP_GOOGLE_AUTH,0.002659
creation_source_SIGNUP,0.001842
creation_source_ORG_INVITE,0.001169


#### The feature of importance shows that org_id is essential in classifying the adapted users, followed by enabled_for_marketing_drip, creation_source from personal project and opted_in_mailing_list. 